In [3]:
%logstop
%logstart -rtq ~/.logs/ml.py append
import seaborn as sns
sns.set()

In [4]:
from static_grader import grader

# ML Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to create effective machine learning models for making predictions. We will be working with nursing home inspection data from the United States, predicting which providers may be fined and for how much.

## Scoring

In this miniproject you will often submit your model's `predict` or `predict_proba` method to the grader. The grader will assess the performance of your model using a scoring metric, comparing it against the score of a reference model. We will use the [average precision score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html). If your model performs better than the reference solution, then you can score higher than 1.0.

**Note:** If you use an estimator that relies on random draws (like a `RandomForestClassifier`) you should set the `random_state=` to an integer so that your results are reproducible. 

## Downloading the data

We can download the data set from Amazon S3:

In [4]:
%%bash
mkdir data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-train.csv -nc -P ./ml-data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-metadata.csv -nc -P ./ml-data

mkdir: cannot create directory ‘data’: File exists
File ‘./ml-data/providers-train.csv’ already there; not retrieving.

File ‘./ml-data/providers-metadata.csv’ already there; not retrieving.



We'll load the data into a Pandas DataFrame. Several columns will become target labels in future questions. Let's pop those columns out from the data, and drop related columns that are neither targets nor reasonable features (i.e. we don't wouldn't know how many times a facility denied payment before knowing whether it was fined).

The data has many columns. We have also provided a data dictionary.

In [5]:
import numpy as np
import pandas as pd

In [6]:
metadata = pd.read_csv('./ml-data/providers-metadata.csv')
metadata.head()

,Variable,Label,Description,Format
0,PROVNUM,Federal Provider Number,Federal Provider Number,6 alphanumeric characters
1,PROVNAME,Provider Name,Provider Name,text
2,ADDRESS,Provider Address,Provider Address,text
3,CITY,Provider City,Provider City,text
4,STATE,Provider State,Provider State,2-character postal abbreviation


In [7]:
data = pd.read_csv('./ml-data/providers-train.csv', encoding='latin1')

fine_counts = data.pop('FINE_CNT')
fine_totals = data.pop('FINE_TOT')
cycle_2_score = data.pop('CYCLE_2_TOTAL_SCORE')

In [8]:
data.head()

,PROVNUM,PROVNAME,ADDRESS,CITY,STATE,ZIP,PHONE,COUNTY_SSA,COUNTY_NAME,BEDCERT,...,CERTIFICATION,CYCLE_1_DEFS,CYCLE_1_NFROMDEFS,CYCLE_1_NFROMCOMP,CYCLE_1_DEFS_SCORE,CYCLE_1_NUMREVIS,CYCLE_1_REVISIT_SCORE,CYCLE_1_TOTAL_SCORE,CYCLE_1_SURVEY_DATE,CYCLE_2_SURVEY_DATE
0,015010,COOSA VALLEY NURSING FACILITY,315 WEST HICKORY STREET,SYLACAUGA,AL,35150,2562495604,600,Talladega,85,...,Medicare and Medicaid,7,7,0,36,1,0,36,2017-04-06,2016-05-26
1,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,350,Jackson,50,...,Medicare and Medicaid,5,5,0,44,1,0,44,2017-03-16,2016-02-04
2,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,360,Jefferson,92,...,Medicare and Medicaid,6,6,0,40,1,0,40,2016-10-20,2015-12-30
3,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY P O BOX 97,MC CALLA,AL,35111,2054776161,360,Jefferson,103,...,Medicare and Medicaid,2,2,0,16,1,0,16,2017-03-09,2016-02-11
4,015016,ATHENS HEALTH AND REHABILITATION LLC,611 WEST MARKET STREET,ATHENS,AL,35611,2562321620,410,Limestone,149,...,Medicare and Medicaid,2,2,0,20,1,0,20,2017-06-01,2016-05-12


In [9]:
fine_totals

0        15259
1            0
2            0
3            0
4            0
         ...  
13887        0
13888        0
13889        0
13890        0
13891    70865
Name: FINE_TOT, Length: 13892, dtype: int64

In [10]:
df=data.copy()
df=df.assign(y=fine_totals)

In [8]:
state_avg=df.groupby('STATE')['y'].agg('mean')
group_avg=dict(state_avg)
group_avg

NameError: name 'df' is not defined

In [12]:
df['STATE']

0        AL
1        AL
2        AL
3        AL
4        AL
         ..
13887    TX
13888    TX
13889    TX
13890    TX
13891    TX
Name: STATE, Length: 13892, dtype: object

In [13]:
df['STATE'].apply(lambda x:group_avg[x])

0        13672.320388
1        13672.320388
2        13672.320388
3        13672.320388
4        13672.320388
             ...     
13887    18147.119744
13888    18147.119744
13889    18147.119744
13890    18147.119744
13891    18147.119744
Name: STATE, Length: 13892, dtype: float64

In [14]:
df['STATE'].map(group_avg)

0        13672.320388
1        13672.320388
2        13672.320388
3        13672.320388
4        13672.320388
             ...     
13887    18147.119744
13888    18147.119744
13889    18147.119744
13890    18147.119744
13891    18147.119744
Name: STATE, Length: 13892, dtype: float64

## Question 1: state_model

A federal agency, Centers for Medicare and Medicaid Services (CMS), imposes regulations on nursing homes. However, nursing homes are inspected by state agencies for compliance with regulations, and fines for violations can vary widely between states.

Let's develop a very simple initial model to predict the amount of fines a nursing home might expect to pay based on its location. Fill in the class definition of the custom estimator, `StateMeanEstimator`, below.

**Note:** When the grader checks your answer, it passes a list of dictionaries to the `predict` method of your estimator, not a DataFrame. This means that your model must work with both data types. You can handle this by adding a test (and optional conversion) in the `predict` method of your custom class, similar to the `ColumnSelectTransformer` given below in Question 2.  

In [8]:
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin

class GroupMeanEstimator(BaseEstimator, RegressorMixin):
    def __init__(self, grouper):
        self.grouper = grouper
    @staticmethod
    def convert_to_df(X):
        if  not isinstance(X,pd.DataFrame):
            return pd.DataFrame(X)
        else:
            return X

    def fit(self, X, y):
        # Use self.group_averages to store the average penalty by group
        X=self.convert_to_df(X)
        state_avg=df.assign(y=fine_totals).groupby(self.grouper)['y'].agg('mean')
        self.group_averages_=dict(state_avg)
        self.average_=y.mean()
        return self

    def predict(self, X):
        # Return a list of predicted penalties based on group of samples in X
        X=self.convert_to_df(X)
        return X[self.grouper].map(self.group_averages_).fillna(self.average_)

After filling in class definition, we can create an instance of the estimator and fit it to the data.

In [16]:
from sklearn.pipeline import Pipeline

state_model = Pipeline([
    ('sme', GroupMeanEstimator(grouper='STATE'))
    ])
state_model.fit(data, fine_totals)

Pipeline(steps=[('sme', GroupMeanEstimator(grouper='STATE'))])

Next we should test that our predict method works.

In [12]:
state_model.predict(data.sample(5))

12770     8054.977612
7259      3050.196429
5887     21437.397468
3483      5626.954545
7630      3490.756839
Name: STATE, dtype: float64

However, what if we have data from a nursing home in a state (or territory) of the US which is not in the training data?

In [17]:
state_model.predict(pd.DataFrame([{'STATE': 'AS'}]))

0    14969.857688
Name: STATE, dtype: float64

Make sure your model can handle this possibility before submitting your model's predict method to the grader.

In [18]:
grader.score.ml__state_model(state_model.predict)

Your score: 1.000


## Question 2: simple_features_model

Nursing homes vary greatly in their business characteristics. Some are owned by the government or non-profits while others are run for profit. Some house a few dozen residents while others house hundreds. Some are located within hospitals and may work with more vulnerable populations. We will try to predict which facilities are fined based on their business characteristics.

We'll begin with columns in our DataFrame containing numeric and boolean features. Some of the rows contain null values; estimators cannot handle null values so these must be imputed or dropped. We will create a `Pipeline` containing transformers that process these features, followed by an estimator.

**Note:** As mentioned above in Question 1, when the grader checks your answer, it passes a list of dictionaries to the `predict` or `predict_proba` method of your estimator, not a DataFrame. This means that your model must work with both data types. For this reason, we've provided a custom `ColumnSelectTransformer` for you to use instead `scikit-learn`'s own `ColumnTransformer`.

In [10]:
simple_cols = ['BEDCERT', 'RESTOT', 'INHOSP', 'CCRC_FACIL', 'SFF', 'CHOW_LAST_12MOS', 'SPRINKLER_STATUS', 'EXP_TOTAL', 'ADJ_TOTAL']
data[simple_cols]

,BEDCERT,RESTOT,INHOSP,CCRC_FACIL,SFF,CHOW_LAST_12MOS,SPRINKLER_STATUS,EXP_TOTAL,ADJ_TOTAL
0,85,74.2,True,False,False,False,True,NaN,NaN
1,50,NaN,True,False,False,False,True,NaN,NaN
2,92,79.8,False,False,False,False,True,3.08015,3.83026
3,103,98.1,False,True,False,False,True,2.83938,3.95709
4,149,119.7,False,False,False,False,True,3.15554,4.07866
...,...,...,...,...,...,...,...,...,...
13887,128,76.3,False,False,False,False,True,NaN,NaN
13888,100,64.5,False,False,False,False,True,3.44863,3.06164
13889,61,NaN,False,False,False,False,True,NaN,NaN
13890,126,105.2,False,False,False,False,True,3.46718,2.60224


In [9]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer  # noqa
 # now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer

In [10]:

simple_cols = ['BEDCERT', 'RESTOT', 'INHOSP', 'CCRC_FACIL', 'SFF', 'CHOW_LAST_12MOS', 'SPRINKLER_STATUS', 'EXP_TOTAL', 'ADJ_TOTAL']

class ColumnSelectTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self
    def transform(self,X):
        if not isinstance(X,pd.DataFrame):
            X=pd.DataFrame(X)
        return X[self.columns]

    
simple_features = Pipeline([
    
   
    ('selector', ColumnSelectTransformer(simple_cols)),
    ('imputer',IterativeImputer()),
    ('scalar',StandardScaler()),
])

**Note:** The assertion below assumes the output of `noncategorical_features.fit_transform` is a `ndarray`, not a `DataFrame`.)

In [11]:
assert data['RESTOT'].isnull().sum() > 0
assert not np.isnan(simple_features.fit_transform(data)).any()

Now combine the `simple_features` pipeline with an estimator in a new pipeline. Fit `simple_features_model` to the data and submit `simple_features_model.predict_proba` to the grader. You may wish to use cross-validation to tune the hyperparameters of your model.

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

simple_features_model = Pipeline([
    ('simple', simple_features),
    ('classifier', GradientBoostingClassifier()),
    # add your estimator here
])

In [19]:
simple_features_model.fit(data, fine_counts > 0)

Pipeline(steps=[('simple',
                 Pipeline(steps=[('selector',
                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                   'RESTOT',
                                                                   'INHOSP',
                                                                   'CCRC_FACIL',
                                                                   'SFF',
                                                                   'CHOW_LAST_12MOS',
                                                                   'SPRINKLER_STATUS',
                                                                   'EXP_TOTAL',
                                                                   'ADJ_TOTAL'])),
                                 ('imputer', IterativeImputer()),
                                 ('scalar', StandardScaler())])),
                ('classifier', GradientBoostingClassifier())])

In [20]:
def positive_probability(model):
    def predict_proba(X):
        return model.predict_proba(X)[:, 1]
    return predict_proba

grader.score.ml__simple_features(positive_probability(simple_features_model))

Your score: 1.154


## Question 3: categorical_features

The `'OWNERSHIP'` and `'CERTIFICATION'` columns contain categorical data. We will have to encode the categorical data into numerical features before we pass them to an estimator. Construct one or more pipelines for this purpose. Transformers such as [LabelEncoder](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.LabelEncoder.html#sklearn.preprocessing.LabelEncoder) and [OneHotEncoder](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder) may be useful, but you may also want to define your own transformers.

If you used more than one `Pipeline`, combine them with a `FeatureUnion`. As in Question 2, we will combine this with an estimator, fit it, and submit the `predict_proba` method to the grader.

In [92]:
from sklearn.preprocessing import OneHotEncoder



class ColumnSelectTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self
    def transform(self,X):
        if not isinstance(X,pd.DataFrame):
            X=pd.DataFrame(X)
        return X[self.columns]
    
    
    
cat_cols=['OWNERSHIP','CERTIFICATION']
categorical_features = Pipeline([
    
    
    ('selector', ColumnSelectTransformer(cat_cols)),
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('encoder',OneHotEncoder(drop='first'))
     ])


In [93]:
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import OrdinalEncoder

class ColumnSelectTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self
    def transform(self,X):
        if not isinstance(X,pd.DataFrame):
            X=pd.DataFrame(X)
        return X[self.columns]
    
    
    
cat_cols=['OWNERSHIP','CERTIFICATION']
categorical_features = Pipeline([
    
    
    ('selector', ColumnSelectTransformer(cat_cols)),
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('encoder',OneHotEncoder(drop='first'))
     ])

In [94]:
assert categorical_features.fit_transform(data).shape[0] == data.shape[0]
assert categorical_features.fit_transform(data).dtype == np.float64
#assert not np.isnan(categorical_features.fit_transform(data)).any()

In [115]:
GradientBoostingClassifier?

As in the previous question, create a model using the `categorical_features`, fit it to the data, and submit its `predict_proba` method to the grader.

In [134]:
categorical_features_model = Pipeline([
    ('categorical', categorical_features),
    ('classifier', GradientBoostingClassifier(learning_rate=0.01))
    # add your estimator here
])

In [135]:
categorical_features_model.fit(data, fine_counts > 0)

Pipeline(steps=[('categorical',
                 Pipeline(steps=[('selector',
                                  ColumnSelectTransformer(columns=['OWNERSHIP',
                                                                   'CERTIFICATION'])),
                                 ('imputer',
                                  SimpleImputer(strategy='most_frequent')),
                                 ('encoder', OneHotEncoder(drop='first'))])),
                ('classifier', GradientBoostingClassifier(learning_rate=0.01))])

In [136]:
grader.score.ml__categorical_features(positive_probability(categorical_features_model))

Your score: 1.055


## Question 4: business_model

Finally, we'll combine `simple_features` and `categorical_features` in a `FeatureUnion`, followed by an estimator in a `Pipeline`. You may want to optimize the hyperparameters of your estimator using cross-validation or try engineering new features (e.g. see [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)). When you've assembled and trained your model, pass the `predict_proba` method to the grader.

In [36]:
from sklearn.pipeline import FeatureUnion
business_features = FeatureUnion([
    ('simple', simple_features),
    ('categorical', categorical_features)
])

In [43]:
from sklearn.ensemble import RandomForestClassifier
business_model = Pipeline([
    ('features', business_features),
    ('classifier',GradientBoostingClassifier())
    # add your estimator here
])

In [44]:
business_model.fit(data, fine_counts > 0)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('simple',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                                                   'RESTOT',
                                                                                                   'INHOSP',
                                                                                                   'CCRC_FACIL',
                                                                                                   'SFF',
                                                                                                   'CHOW_LAST_12MOS',
                                                                                                   'SPRINKLER_STATUS',
                                                                                                   'EXP_TOTAL',
                                                                                                   'ADJ_TOTAL'])),
                                                                 ('imputer',
                                                                  IterativeImputer()),
                                                                 ('scalar',
                                                                  StandardScaler())])),
                                                ('categorical',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnSelectTransformer(columns=['OWNERSHIP',
                                                                                                   'CERTIFICATION'])),
                                                                 ('imputer',
                                                                  SimpleImputer(strategy='most_frequent')),
                                                                 ('encoder',
                                                                  OneHotEncoder())]))])),
                ('classifier', GradientBoostingClassifier())])

In [39]:
from sklearn import set_config
set_config(display='diagram')
business_model


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('simple',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                                                   'RESTOT',
                                                                                                   'INHOSP',
                                                                                                   'CCRC_FACIL',
                                                                                                   'SFF',
                                                                                                   'CHOW_LAST_12MOS',
                                                                                                   'SPRINKLER_STATUS',
                                                                                                   'EXP_TOTAL',
                                                                                                   'ADJ_TOTAL'])),
                                                                 ('imputer',
                                                                  IterativeImputer()),
                                                                 ('scalar',
                                                                  StandardScaler())])),
                                                ('categorical',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnSelectTransformer(columns=['OWNERSHIP',
                                                                                                   'CERTIFICATION'])),
                                                                 ('imputer',
                                                                  SimpleImputer(strategy='most_frequent')),
                                                                 ('encoder',
                                                                  OneHotEncoder())]))])),
                ('classifier', RandomForestClassifier())])

In [40]:
categorical_features

Pipeline(steps=[('selector',
                 ColumnSelectTransformer(columns=['OWNERSHIP',
                                                  'CERTIFICATION'])),
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('encoder', OneHotEncoder())])

In [41]:
simple_features_model

Pipeline(steps=[('simple',
                 Pipeline(steps=[('selector',
                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                   'RESTOT',
                                                                   'INHOSP',
                                                                   'CCRC_FACIL',
                                                                   'SFF',
                                                                   'CHOW_LAST_12MOS',
                                                                   'SPRINKLER_STATUS',
                                                                   'EXP_TOTAL',
                                                                   'ADJ_TOTAL'])),
                                 ('imputer', IterativeImputer()),
                                 ('scalar', StandardScaler())])),
                ('classifier', GradientBoostingClassifier())])

In [45]:
grader.score.ml__business_model(positive_probability(business_model))

Your score: 1.034


## Question 5: survey_results

Surveys reveal safety and health deficiencies at nursing homes that may indicate risk for incidents (and penalties). CMS routinely makes surveys of nursing homes. Build a model that combines the `business_features` of each facility with its cycle 1 survey results, as well as the time between the cycle 1 and cycle 2 survey to predict the cycle 2 total score.

First, let's create a transformer to calculate the difference in time between the cycle 1 and cycle 2 surveys.

In [53]:
df_dates=data[['CYCLE_1_SURVEY_DATE','CYCLE_2_SURVEY_DATE']]
df_dates

,CYCLE_1_SURVEY_DATE,CYCLE_2_SURVEY_DATE
0,2017-04-06,2016-05-26
1,2017-03-16,2016-02-04
2,2016-10-20,2015-12-30
3,2017-03-09,2016-02-11
4,2017-06-01,2016-05-12
...,...,...
13887,2017-08-10,2016-10-05
13888,2017-06-22,2016-07-21
13889,2017-09-28,2016-12-14
13890,2017-11-09,2016-12-13


In [54]:
df_dates.dtypes

CYCLE_1_SURVEY_DATE    object
CYCLE_2_SURVEY_DATE    object
dtype: object

In [55]:
def fxn(df,col):
    df[col]=pd.to_datetime(df[col],format='%Y-%m-%d')

In [58]:
fxn(df_dates,'CYCLE_1_SURVEY_DATE')
fxn(df_dates,'CYCLE_2_SURVEY_DATE')

<ipython-input-55-24bdf1e62776>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col]=pd.to_datetime(df[col],format='%Y-%m-%d')


In [14]:
delta_t=df_dates['CYCLE_1_SURVEY_DATE']-df_dates['CYCLE_2_SURVEY_DATE']
delta_t

0       315 days
1       406 days
2       295 days
3       392 days
4       385 days
          ...   
13887   309 days
13888   336 days
13889   288 days
13890   331 days
13891    98 days
Length: 13892, dtype: timedelta64[ns]

In [19]:
delta_t.dt.total_seconds().values.reshape(-1,1)

array([[27216000.],
       [35078400.],
       [25488000.],
       ...,
       [24883200.],
       [28598400.],
       [ 8467200.]])

In [ ]:
class TimedeltaTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, t1_col, t2_col):
        self.t1_col = t1_col
        self.t2_col = t2_col

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        t2_col_values = pd.to_datetime(X[self.t2_col])
        t1_col_values = pd.to_datetime(X[self.t1_col])
        return (t1_col_values - t2_col_values).dt.days.values.reshape(-1,1)
        

In [73]:
cycle_1_date = 'CYCLE_1_SURVEY_DATE'
cycle_2_date = 'CYCLE_2_SURVEY_DATE'
time_feature = TimedeltaTransformer(cycle_1_date, cycle_2_date)

In the cell below we'll collect the cycle 1 survey features.

In [74]:
cycle_1_cols = ['CYCLE_1_DEFS', 'CYCLE_1_NFROMDEFS', 'CYCLE_1_NFROMCOMP',
                'CYCLE_1_DEFS_SCORE', 'CYCLE_1_NUMREVIS',
                'CYCLE_1_REVISIT_SCORE', 'CYCLE_1_TOTAL_SCORE']
cycle_1_features =Pipeline([('selector',ColumnSelectTransformer(cycle_1_cols))])

In [91]:
from sklearn.ensemble import RandomForestRegressor

from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import PolynomialFeatures

survey_model = Pipeline([
    ('features', FeatureUnion([
        ('business', business_features),
        ('survey', cycle_1_features),
        ('time', time_feature)
    ])),
    ('poly',PolynomialFeatures(degree=2,include_bias=False)),
    ('red',TruncatedSVD(n_components=50)),
    ('model',RandomForestRegressor(n_estimators=100,min_samples_leaf=50,random_state=0))
    # add your estimator here
])

In [71]:
class TimedeltaTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, t1_col, t2_col):
        self.t1_col = t1_col
        self.t2_col = t2_col

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        t2_col_values = pd.to_datetime(X[self.t2_col])
        t1_col_values = pd.to_datetime(X[self.t1_col])
        return (t1_col_values - t2_col_values).dt.days.values.reshape(-1,1)
        

In [92]:
survey_model.fit(data, cycle_2_score.astype(int))

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('business',
                                                 FeatureUnion(transformer_list=[('simple',
                                                                                 Pipeline(steps=[('selector',
                                                                                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                                                                                   'RESTOT',
                                                                                                                                   'INHOSP',
                                                                                                                                   'CCRC_FACIL',
                                                                                                                                   'SFF',
                                                                                                                                   'CHOW_LAST_12MOS',
                                                                                                                                   'SPRINKLER_STATUS',
                                                                                                                                   'EXP_TOTAL',
                                                                                                                                   'ADJ_TOTAL'])),
                                                                                                 ('imputer',
                                                                                                  IterativeImputer()),
                                                                                                 ('scalar',
                                                                                                  StandardScaler())])),
                                                                                ('ca...
                                                                                                   'CYCLE_1_NFROMCOMP',
                                                                                                   'CYCLE_1_DEFS_SCORE',
                                                                                                   'CYCLE_1_NUMREVIS',
                                                                                                   'CYCLE_1_REVISIT_SCORE',
                                                                                                   'CYCLE_1_TOTAL_SCORE']))])),
                                                ('time',
                                                 TimedeltaTransformer(t1_col='CYCLE_1_SURVEY_DATE',
                                                                      t2_col='CYCLE_2_SURVEY_DATE'))])),
                ('poly', PolynomialFeatures(include_bias=False)),
                ('red', TruncatedSVD(n_components=50)),
                ('model',
                 RandomForestRegressor(min_samples_leaf=50, random_state=0))])

In [93]:
grader.score.ml__survey_model(survey_model.predict)

Your score: 1.108


*Copyright &copy; 2021 WorldQuant University. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*